In [ ]:
## Class
## 1. 객체 모델링의 수단
## 2. ADT(abstract data type) - 추상데이터 타입

## class안에 정의되는 내용은 크게 3가지
## 상태값 저장을 위한 변수(field, member variable, property)
## 수행하는 작업을 위한 함수(method, member function, method)
## 정의된 class정보를 바탕으로 일정 메모리를 확보
## => 이런 확보된 메모리 공간 -> 인스턴스, 객체
## 이런 객체를 생성하기 위해 생성자가 호출되어야 해요!

## 학생 3명이 있어요
## 각 학생이 가지는 정보는 국어, 영어, 수학 점수
## 평균, 총점 결과를 알고 싶어요
class Student:
    
    # Constructor
    def __init__(self,s_name,s_kor,s_eng,s_math):
            # property를 생성하려면 self를 이용
            self.student_name = s_name
            self.kor = s_kor
            self.eng = s_eng
            self.math = s_math
        
     # method
    def get_total(self):
        self.total = self.kor+self.eng+self.math
        return self.total
        
stu1 = Student("홍길동", 10,20,30)
print(stu1.get_total())

In [1]:
## Enselble을 이용한 MNIST
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import warnings
import pandas as pd
import numpy as np
warnings.filterwarnings(action = "ignore")

## Graph 초기화
tf.reset_default_graph()

## Class Define
class CnnModel:
    # constructor
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self.build_net()
        
    # Model graph 생성하는 mothod
    def build_net(self):
        with tf.variable_scope(self.name):
            self.X = tf.placeholder(shape=[None,784], dtype=tf.float32)
            self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
            self.drop_rate = tf.placeholder(dtype=tf.float32)
            X_img=tf.reshape(self.X, [-1,28,28,1])

#             filter1 = tf.Variable(tf.random_normal([3,3,1,32]))
#             L1 = tf.nn.conv2d(X_img,filter1,strides=[1,1,1,1], padding="SAME")
#             L1 = tf.nn.relu(L1)
#             L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME") # kernel 사이즈 가로2 세로2 의미

            L1 = tf.layers.conv2d(inputs=X_img, filters=3, kernel_size=[3,3], padding="SAME",strides=1, activation=tf.nn.relu)
            L1 = tf.layers.max_pooling2d(inputs = L1, pool_size=[2,2], padding="SAME", strides=2) # ksize를 poo_size로 바꿔ㅜ줄거

            L2 = tf.layers.conv2d(inputs=L1, filters=6, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
            L2 = tf.layers.max_pooling2d(inputs = L2, pool_size=[2,2], padding="SAME", strides=2) # maxpooling

#             print(L2.shape)
            L2 = tf.reshape(L2,[-1,7*7*6])

            W1 = tf.get_variable("weight1", shape=[7*7*6, 10], initializer=tf.contrib.layers.xavier_initializer()) # 컬럼의갯수, 아웃풋갯수(임의)
            b1 = tf.Variable(tf.random_normal([10]), name="bias1")
            _layer1 = tf.nn.relu(tf.matmul(L2,W1) + b1)
            layer1 = tf.layers.dropout(_layer1, rate=self.drop_rate)

            W2 = tf.get_variable("weight2", shape=[10, 10], initializer=tf.contrib.layers.xavier_initializer()) # 컬럼의갯수, 아웃풋갯수(임의)
            b2 = tf.Variable(tf.random_normal([10]), name="bias2")
            
        logits = tf.matmul(layer1, W2) + b2
        self.H = tf.nn.relu(logits)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=self.Y))
        self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)
        
    # Model 학습
    def train_net(self, train_x_data, train_y_data):
        
        # session object & initialization

        training_epoch = 15 # 니가학습 몇번시킬래
        batch_size = 50 # 데이터를 몇등분해서 학습시킬래 # 환경이좋으면 작게 넣어서 한번에 여러번 학습

        for step in range(training_epoch):
            num_of_iteration = int(train_x_data.shape[0] / batch_size)
            cost_val = 0
    
            for i in range(num_of_iteration):
                batch_x, batch_y = train_x_data[i*batch_size:(i+1)*batch_size],train_y_data[i*batch_size:(i+1)*batch_size]
                _, cost_val = sess.run([self.train, self.cost], feed_dict={self.X: batch_x, self.Y: batch_y, self.drop_rate: 0.5})

            if step %2 == 0:
                print(cost_val)
    
    # Model의 정확도 측정
    def get_accuracy(self, test_x_data, test_y_data):
        self.predict = tf.argmax(self.H,1)
        correct = tf.equal(self.predict, tf.argmax(self.Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
        print("Accuracy: {}".format(sess.run(accuracy, feed_dict = {self.X: test_x_data, self.Y: test_y_data, self.drop_rate:0.5})))
    
    # Model의 Prediction
    def get_prediction(self, x_data, rate):
        result = sess.run(self.predict, feed_dict={self.X:x_data, self.drop_rate: rate})
        df = pd.DataFrame({
            'ImageId': [i for i in range(1,28001)],
            'Label': result
        })
        df.to_csv('./data/kaggle/submission2.csv', index=False)
        dispalay(df)

## 1. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)
## sessino 생성
sess = tf.Session()
## 2.Model의 개수를 지정하고 생성
num_of_model = 5
cnn_models = [CnnModel(sess,"Model" + str(x)) for x in range(num_of_model)]
for model in cnn_models:
    sess.run(tf.global_variables_initializer())
    model.train_net(mnist.train.images, mnist.train.labels)
    model.get_accuracy(mnist.test.images, mnist.test.labels)
    model.get_prediction(mnist.test.images,0.3)
    

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dropout instead.


KeyboardInterrupt: 